In [65]:
# importing the neccessary libraries
import numpy as np
from time import time
from time import sleep
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
num_shares = []
percent_share = []
share_holders = []
share_detail = []
director = []
names = []
# creating page numbers from zero and equally spaced by 15 as detected on the target page url and iterating through pages
pages = [str(i) for i in np.arange(0,45,15)]
start_time = time()
request = 0
for page in pages:
    # we input the target url as search.
    search_url ='https://app.companiesoffice.govt.nz/companies/app/ui/pages/companies/search?q=&entityTypes=ALL&entityStatusGroups=REGISTERED&incorpFrom=&incorpTo=&addressTypes=ALL&addressKeyword=East+Tamaki&start='+page +'&limit=15&sf=&sd=&advancedPanel=true&mode=advanced&securityCookieName=&securityCookieName='
    #open chrome browser and get the target url then use the wait instance of the driver.
    driver = webdriver.Chrome()
    driver.get(search_url)
    driver.implicitly_wait(100)
    #for monitoring, we increase the request by 1 on every get request made by the driver 
    #and calculate the frequency of request
    request +=1
    elapsed_time = time() - start_time
    print('request: {}; frequency: {} request/s'.format(request, request/elapsed_time))
    #we break the process if the request is more than we want- in this case, 50.
    if request > 50:
        break
    #on our page, the distinctive javascript element that we want clicked has a class name entityName.
    #we count the number of that element on a page and itereate the clicking by the number
    num_links = len(driver.find_elements_by_class_name("entityName"))
    for i in range(num_links):
        botton = driver.find_elements_by_class_name("entityName")[i]
        botton.click()
        #We are actually interested in scraping the shareholders page, so we click that too and get the html of the page
        shareholding = driver.find_element_by_id('shareholdingsTab')
        shareholding.click()
        sleep(10)
        page_source = driver.page_source
        #we pass the html of the page to BeautifulSoup for parsing and extraction of required data
        # we append each data to their respective list
        page_soup = BeautifulSoup(page_source, 'html.parser')
        allocation = page_soup.find_all('div', class_='allocationDetail')
        for allot in allocation:
            comp_name = page_soup.find('div', class_='entity').h1.text.strip()
            share_no = allot.find('span', class_='allocationNumber').text
            num_shares.append(share_no)
            share_p = allot.find('span', class_='shareLabel').text.replace('\xa0','').replace('(','').replace(')','').replace('shares','')
            percent_share.append(share_p)
            name = allot.find_all('div',class_='labelValue')[0].text.strip()
            share_holders.append(name)
            detail = allot.find_all('div',class_='labelValue')[1].text.strip()
            share_detail.append(detail)
            # we use an exception here because not all shareholders page have position of shareholder.
            try:
                position = allot.find('span', class_= 'label directorLabel').text
                director.append(position)
            except Exception:
                position = None
                director.append(position)
            #In order for comp_name to be same lenght with the rest of the list, we multiply by the len of number of shares
            comp_name = [comp_name]*len(num_shares)
            names.append(comp_name)
            #return the browser to the last 2 history and continue iteration
            driver.execute_script("window.history.go(-2)")
#finally close brower on the completion of each page extraction
driver.quit()

request: 1; frequency: 0.03400863224746306 request/s
request: 2; frequency: 0.004913722913778465 request/s
request: 3; frequency: 0.0039597785907060385 request/s


In [66]:
import pandas as pd
comp_df = pd.DataFrame(list(zip(names,share_holders, percent_share,num_shares,share_detail,director)), columns= 
             ["company", "shareholder_name", "percent_share","allocation_no", "share_detail", "position"])


In [67]:
comp_df['company'] = comp_df['company'].astype(str).str.replace('[','').str.replace(']','')

In [68]:
comp_df

,company,shareholder_name,percent_share,allocation_no,share_detail,position
0,'NAND CORPORATION LIMITED (2466968) Registered',Sarvada NAND,100.00%,1,"40 Redoubt Road, Manukau City\n, New Zealand",None
1,'A 2 Z NZ POWERCOM LIMITED (836853) Registere...,Rosa Josephine DEVATHASAN,50.00%,1,"443 East Tamaki Road, East Tamaki, Auckland, 2...",None
2,'A 2 Z NZ POWERCOM LIMITED (836853) Registere...,Senthiya DEVATHASAN,50.00%,2,"443 East Tamaki Road, East Tamaki, Auckland, 2...",Director:
3,"""ABDUL'S TRUCK AND TRAILER LIMITED (5365241) ...",Abdul Zuned KHAN,100.00%,1,"712a Massey Road, Mangere, Auckland, 2022\n, N...",Director:
4,"""ABDUL'S TRUCK AND TRAILER WORKSHOP LIMITED (5...",Abdul Azeem KHAN,100.00%,1,"712a Massey Road, Mangere, Auckland, 2022\n, N...",Director:
5,'APE SCAFFOLDING COMPANY NZ LIMITED (5423301) ...,Zongyu WEI,70.00%,1,"8 Simon Owen Place, Golflands, Auckland, 2013\...",Director:
6,'APE SCAFFOLDING COMPANY NZ LIMITED (5423301) ...,Yiqun WANG,30.00%,2,"38b Waterloo Street, Howick, Auckland, 2014\n,...",None
7,'AUCKLAND MAG & TYRE WORLD LIMITED (1724825) ...,Sarvada NAND,100.00%,1,"1-45 Pah Road, Papatoetoe, Auckland\n,",None
8,'BCG WASTE MANAGEMENT LIMITED (3035602) Regis...,WASTE MANAGEMENT NZ LIMITED,100.00%,1,"318 East Tamaki Road, East Tamaki, Auckland, 2...",None
9,"'BUDGET BINS LIMITED (686949) Registered', 'B...",WASTE MANAGEMENT NZ LIMITED,100.00%,1,"318 East Tamaki Road, East Tamaki, Auckland, 2...",None
